In [1]:
# put this in it's own codeblock because it'll keep 
# running otherwise and eventually move to the root dir lol
%cd ..

/Users/angus/code/jupyter/env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/angus/code/jupyter/notebooks/20240907, vg


# ABS API

- [Swagger docs](https://api.gov.au/assets/APIs/abs/DataAPI.openapi.html#/Get%20Data/GetData)
- [Metadata and data](https://www.abs.gov.au/about/data-services/application-programming-interfaces-apis/data-api-user-guide/using-api#get-metadata)

In [12]:
import logging
import json
import os
from pprint import pprint
from lib.service.http.client_session import ClientSession
from lib.service.api_client.abs import AbsClient

logging.basicConfig(level=logging.DEBUG)

# Census 2021, T17 Dwelling structure and number of bedrooms by
# number of persons usually resident for group households, Time
# Series, Main Statistical Areas Level 2 and up (SA2+)
BEDROOMS_ID = 'C21_T17_SA2'

async with ClientSession.create() as session:
    abs_http = AbsClient(session)
    
    with open(f'./_fixtures/absapi_data_{BEDROOMS_ID}-*-_T-AUS-*-*.json', 'w') as f:
        dataflow = await abs_http.find_dataflow_meta(BEDROOMS_ID, references='contentconstraint')
        data_key = [None, '_T', 'AUS', None, None]
        json.dump(await abs_http.get_data(dataflow, data_key, None, 'AllDimensions'), f, indent=2)
        
    with open('./_fixtures/absapi_meta_all_dataflow.json', 'w') as f:
        json.dump(await abs_http.get_structure_meta('dataflow', 'all', 'allcompletestubs'), f, indent=2)
    
    with open(f'./_fixtures/absapi_meta_{BEDROOMS_ID}_dataflow_with_all_refs.json', 'w') as f:
        json.dump(await abs_http.get_structure_meta('dataflow', BEDROOMS_ID, 'full', references='all'), f, indent=2)
    
    with open(f'./_fixtures/absapi_meta_{BEDROOMS_ID}_datastructure.json', 'w') as f:
        json.dump(await abs_http.get_structure_meta('datastructure', BEDROOMS_ID, 'full'), f, indent=2)
    

async with ClientSession.create() as session:
    abs_http = AbsClient(session)
    dataflow = await abs_http.find_dataflow_meta(BEDROOMS_ID, references='contentconstraint')
    datastucture = await abs_http.find_datastructure_meta(BEDROOMS_ID)# startPeriod=2011&dimensionAtObservation=AllDimensions
    data_bedrooms = await abs_http.get_data_all_dimensions(
        dataflow, 
        [None, None, 'AUS', None, None], 
        start_period='2011',
        detail='full')
    pprint(data_bedrooms)

    


DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/dataflow/ABS/C21_T17_SA2?detail=full&references=contentconstraint
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/data/ABS,C21_T17_SA2,1.0.0/._T.AUS..?dimensionAtObservation=AllDimensions
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/dataflow/ABS/all?detail=allcompletestubs
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/dataflow/ABS/C21_T17_SA2?detail=full&references=all
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/datastructure/ABS/C21_T17_SA2?detail=full
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/dataflow/ABS/C21_T17_SA2?detail=full&references=contentconstraint
DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/datastructure/ABS/C21_T17_SA2?detail=full
DEBUG:li

[Observation.AllDimensions(count=6503, dimensions=['_T', '5', 'AUS', 'AUS', 'AUS', '2011']),
 Observation.AllDimensions(count=9611, dimensions=['_T', '5', 'AUS', 'AUS', 'AUS', '2016']),
 Observation.AllDimensions(count=8735, dimensions=['_T', '5', 'AUS', 'AUS', 'AUS', '2021']),
 Observation.AllDimensions(count=20891, dimensions=['_T', '4', 'AUS', 'AUS', 'AUS', '2011']),
 Observation.AllDimensions(count=26548, dimensions=['_T', '4', 'AUS', 'AUS', 'AUS', '2016']),
 Observation.AllDimensions(count=26086, dimensions=['_T', '4', 'AUS', 'AUS', 'AUS', '2021']),
 Observation.AllDimensions(count=2961, dimensions=['_T', '6', 'AUS', 'AUS', 'AUS', '2011']),
 Observation.AllDimensions(count=6192, dimensions=['_T', '6', 'AUS', 'AUS', 'AUS', '2016']),
 Observation.AllDimensions(count=4540, dimensions=['_T', '6', 'AUS', 'AUS', 'AUS', '2021']),
 Observation.AllDimensions(count=231219, dimensions=['_T', '2', 'AUS', 'AUS', 'AUS', '2011']),
 Observation.AllDimensions(count=245353, dimensions=['_T', '2', '

In [3]:
data_bedrooms[0]._table.raw

{'name': 'Census 2021, T17 Dwelling structure and number of bedrooms by number of persons usually resident for group households, Time Series, Main Statistical Areas Level 2 and up (SA2+)',
 'names': {'en': 'Census 2021, T17 Dwelling structure and number of bedrooms by number of persons usually resident for group households, Time Series, Main Statistical Areas Level 2 and up (SA2+)'},
 'description': "Dwelling structure and number of bedrooms by number of persons usually resident for group households time series data for Main Statistical Areas Level 2 and up (SA2+), 2021 Census, Count of occupied private dwellings containing group households.\nThis table is based on place of enumeration.\nIncludes up to three residents who were temporarily absent on Census Night.\nExcludes 'Family', 'Lone person', 'Visitors only' and 'Other non-classifiable' households.\nPlease note that there are small random adjustments made to all cell values to protect the confidentiality of data. These adjustments 

In [4]:
[
    (observation.count, list(observation.dimensions.values())) 
    for observation in data_bedrooms
]

[(6503, ['_T', '5', 'AUS', 'AUS', 'AUS', '2011']),
 (9611, ['_T', '5', 'AUS', 'AUS', 'AUS', '2016']),
 (8735, ['_T', '5', 'AUS', 'AUS', 'AUS', '2021']),
 (20891, ['_T', '4', 'AUS', 'AUS', 'AUS', '2011']),
 (26548, ['_T', '4', 'AUS', 'AUS', 'AUS', '2016']),
 (26086, ['_T', '4', 'AUS', 'AUS', 'AUS', '2021']),
 (2961, ['_T', '6', 'AUS', 'AUS', 'AUS', '2011']),
 (6192, ['_T', '6', 'AUS', 'AUS', 'AUS', '2016']),
 (4540, ['_T', '6', 'AUS', 'AUS', 'AUS', '2021']),
 (231219, ['_T', '2', 'AUS', 'AUS', 'AUS', '2011']),
 (245353, ['_T', '2', 'AUS', 'AUS', 'AUS', '2016']),
 (253602, ['_T', '2', 'AUS', 'AUS', 'AUS', '2021']),
 (321010, ['_T', '_T', 'AUS', 'AUS', 'AUS', '2011']),
 (354911, ['_T', '_T', 'AUS', 'AUS', 'AUS', '2016']),
 (361822, ['_T', '_T', 'AUS', 'AUS', 'AUS', '2021']),
 (59429, ['_T', '3', 'AUS', 'AUS', 'AUS', '2011']),
 (67215, ['_T', '3', 'AUS', 'AUS', 'AUS', '2016']),
 (68868, ['_T', '3', 'AUS', 'AUS', 'AUS', '2021']),
 (126316, ['11', '2', 'AUS', 'AUS', 'AUS', '2011']),
 (126536

In [5]:
from pprint import pprint
from lib.service.http.client_session import ClientSession
from lib.service.api_client.abs import AbsClient
async with ClientSession.create() as session:
    abs_http = AbsClient(session)
    dataflows = await abs_http.get_dataflow_meta()

pprint([d for d in dataflows if 'dwelling' in d.name.lower() and 'sa1' in d.name.lower()])

DEBUG:lib.service.api_client.abs.client.AbsClient:GET https://data.api.abs.gov.au/rest/dataflow/ABS/all?detail=allcompletestubs


[DataFlowMeta(id='ABS_CENSUS2011_B29_SA1_SA',
              version='1.0.0',
              name='B29 Number of Motor Vehicles by Dwellings (SA1 SA)',
              desc='SA1 based data for Household Composition By Number Of '
                   'Persons Usually Resident Basic Community Profile Table '
                   'B30, for 2011 Census. Unit of measure: Count of all '
                   'persons on Census night, based on place of usual '
                   'residence. Excludes Overseas Visitors. Geographic '
                   'coverage: ASGS2011 Australia/SA1. Catalogue number: 2916.0',
              params=[DataFlowParam(id='REGION',
                                    default_value='40101100101'),
                      DataFlowParam(id='FREQUENCY',
                                    default_value='A'),
                      DataFlowParam(id='TIME_PERIOD_START',
                                    default_value='2011')],
              layout_col=['FPD'],
              layout_r

In [6]:
df = next(d for d in dataflows if d.id == 'ABS_CENSUS2011_B36_SA1_SA')
df.raw

{'id': 'ABS_CENSUS2011_B36_SA1_SA',
 'links': [{'href': 'http://data.api.abs.gov.au/rest/dataflow/ABS/ABS_CENSUS2011_B36_SA1_SA/1.0.0',
   'rel': 'external'}],
 'version': '1.0.0',
 'agencyID': 'ABS',
 'isExternalReference': True,
 'isFinal': True,
 'name': 'B36 Dwelling Structure by Number of Bedrooms (SA1 SA)',
 'names': {'en': 'B36 Dwelling Structure by Number of Bedrooms (SA1 SA)'},
 'description': 'SA1 based data for Dwelling Structure By Number Of Bedrooms Basic Community Profile Table B36, for 2011 Census. Unit of measure: Count of all persons on Census night, based on place of usual residence. Excludes Overseas Visitors. Geographic coverage: ASGS2011 Australia/SA1. Catalogue number: 2916.0',
 'descriptions': {'en': 'SA1 based data for Dwelling Structure By Number Of Bedrooms Basic Community Profile Table B36, for 2011 Census. Unit of measure: Count of all persons on Census night, based on place of usual residence. Excludes Overseas Visitors. Geographic coverage: ASGS2011 Austra